In [1]:
import tensorflow as tf
all_files = tf.data.Dataset.list_files("runs2\\*\\.fullpic\\*.png")
all_files = list(all_files.as_numpy_iterator())

In [20]:
f = all_files[0]
f, f.decode()

(b'runs2\\00495\\.fullpic\\0000000031+system.png',
 'runs2\\00495\\.fullpic\\0000000031+system.png')

In [2]:
len(all_files)

18145

In [2]:
from PIL import Image
import models.model_experiemnt_lab as teacher
import importlib
importlib.reload(teacher)
imgs = [Image.open(f).resize((320,200)) for f in all_files[:10]]
preds = teacher.many_preds(imgs)
preds.shape

(10, 6, 2)

In [21]:
from math import ceil
class AllFilesSequence(tf.keras.utils.Sequence):
    def __init__(self, batch_size) -> None:
        self.batch_size = batch_size
    def __len__(self):
        return ceil(len(all_files)/self.batch_size)
    def __getitem__(self, idx):
        low = idx*self.batch_size
        high = min(low + self.batch_size, len(all_files))
        batch_files = all_files[low:high]
        batch_files = [i.decode() for i in batch_files]
        imgs = [Image.open(f).resize((320,200)) for f in batch_files]
        preds = teacher.many_preds(imgs)
        return batch_files, preds


In [29]:
import pandas as pd
from tqdm.notebook import tqdm
import threading
stop_predicting = False
def make_all_preds():
    allFilesSequence=AllFilesSequence(100)
    df = None
    for files, preds in tqdm(allFilesSequence):
    # for files, preds in tqdm([allFilesSequence[i] for i in range(3)]):
        if stop_predicting: break
        avoids = preds[:,:,0]
        df_ = pd.DataFrame(avoids.T, columns=map(str, files))
        df = df_ if df is None else pd.concat([df,df_], axis=1)
        df.to_csv('teacher_preds.csv')
teacher_thread = threading.Thread(target=make_all_preds)
teacher_thread.start()

  0%|          | 0/182 [00:00<?, ?it/s]

In [28]:
stop_predicting=True

  0%|          | 0/3 [00:00<?, ?it/s]

In [114]:
dfr = pd.read_csv("teacher_preds.csv")
dfr = dfr.iloc[:, 1:]
dfr.shape

(6, 18144)

In [69]:
import numpy as np
dfr_np = dfr.to_numpy()
dfr_stat = pd.DataFrame(
    [np.argmax(dfr_np, 0), np.argmin(dfr_np, 0)],
    columns=dfr.columns,
    index=['argmax', 'argmin']
)
dfr_stat.shape

(2, 12600)

In [70]:
stat = dfr_stat.T.sort_values(['argmax', 'argmin'])
stat.head()

,argmax,argmin
runs2\00516\.fullpic\0000000073+system.png,0,1
runs2\00394\.fullpic\0000000029+system.png,0,1
runs2\00479\.fullpic\0000000003+system.png,0,1
runs2\00270\.fullpic\0000000037+system.png,0,1
runs2\00183\.fullpic\0000000038+system.png,0,1


In [75]:
argmax_vals = stat['argmax'].unique()
argmin_vals = stat['argmin'].unique()
argmax_vals,argmin_vals

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([1, 2, 3, 4, 5, 0], dtype=int64))

In [78]:
amag = stat.groupby('argmax')
amig = stat.groupby('argmin')


In [79]:
amag.count()

,argmin
argmax,
0,2085
1,1862
2,2284
3,2005
4,1787
5,2577


In [80]:
amig.count()

,argmax
argmin,
0,2074
1,2436
2,1964
3,2039
4,2677
5,1410


In [77]:
1203/6

200.5

In [83]:
take = [amag.get_group(i).sample(201) for i in argmax_vals]
take = pd.concat(take, axis=0)

In [84]:
take.groupby('argmin').count()

,argmax
argmin,
0,194
1,226
2,179
3,194
4,272
5,141


In [86]:
take_files = take.index

In [87]:
take_files

Index(['runs2\00276\.fullpic\0000000032+system.png',
       'runs2\00443\.fullpic\0000000007+system.png',
       'runs2\00490\.fullpic\0000000002+system.png',
       'runs2\00497\.fullpic\0000000023+system.png',
       'runs2\00370\.fullpic\0000000014+system.png',
       'runs2\00347\.fullpic\0000000012+system.png',
       'runs2\00026\.fullpic\0000000000+system.png',
       'runs2\00340\.fullpic\0000000055+system.png',
       'runs2\00274\.fullpic\0000000034+system.png',
       'runs2\00163\.fullpic\0000000132+system.png',
       ...
       'runs2\00084\.fullpic\0000000035+system.png',
       'runs2\00170\.fullpic\0000000116+system.png',
       'runs2\00197\.fullpic\0000000021+system.png',
       'runs2\00257\.fullpic\0000000000+system.png',
       'runs2\00296\.fullpic\0000000005+system.png',
       'runs2\00012\.fullpic\0000000015+system.png',
       'runs2\00379\.fullpic\0000000010+system.png',
       'runs2\00349\.fullpic\0000000077+system.png',
       'runs2\00346\.fullpic\000000

In [ ]:
from math import ceil
def pull_balanced_preds(dfr, count):
    dfr_np = dfr.to_numpy()
    dfr_stat = pd.DataFrame(
        [np.argmax(dfr_np, 0), np.argmin(dfr_np, 0)],
        columns=dfr.columns,
        index=['argmax', 'argmin']
    )
    stat = dfr_stat.T 
    argmax_vals = stat['argmax'].unique()
    count = ceil(count/len(argmax_vals))
    amag = stat.groupby('argmax')
    take = [amag.get_group(i).sample(count) for i in argmax_vals]
    take = pd.concat(take, axis=0)
    return take


In [116]:
files = pd.DataFrame([i.split("\\") for i in dfr.columns.sort_values()])
grouped = files.groupby(1)
groups = [grouped.get_group(i).sort_values(3).iloc[-4:] for i in files[1].unique()]
ends = pd.concat(groups)
# ['\\'.join(i) for i in ends.to_numpy()]
ends

,0,1,2,3
1,runs2,00000,.fullpic,0000000001+system.png
2,runs2,00000,.fullpic,0000000002+system.png
3,runs2,00000,.fullpic,0000000003+system.png
4,runs2,00000,.fullpic,0000000004+system.png
37,runs2,00001,.fullpic,0000000032+system.png
...,...,...,...,...
18112,runs2,00555,.fullpic,0000000013+system.png
18140,runs2,00556,.fullpic,0000000027+system.png
18141,runs2,00556,.fullpic,0000000028+system.png
18142,runs2,00556,.fullpic,0000000029+system.png
